In [1]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics, DataFrames
#read train and validation data and split it into X and y
X_train_solar=Matrix(DataFrame(CSV.File("X_train_solar.csv")))
y_train_solar=Matrix(DataFrame(CSV.File("y_train_solar.csv")))
X_train_wind=Matrix(DataFrame(CSV.File("X_train_wind.csv")))
y_train_wind=Matrix(DataFrame(CSV.File("y_train_wind.csv")))

X_valid_solar=Matrix(DataFrame(CSV.File("X_valid_solar.csv")))
y_valid_solar=Matrix(DataFrame(CSV.File("y_valid_solar.csv")))
X_valid_wind=Matrix(DataFrame(CSV.File("X_valid_wind.csv")))
y_valid_wind=Matrix(DataFrame(CSV.File("y_valid_wind.csv")));

In [3]:
function lasso(X,y,lambda=0.25)
    
    #Build model
    model = Model(Gurobi.Optimizer)#we have defined the model, pass Gurobi optimizer into the model
    set_optimizer_attribute(model,"OutputFlag",0)
    
    #Insert variables
    n, p = size(X)
    
    @variable(model, beta[j=1:p])
    @variable(model, beta_abs[j=1:p])
    
    
    #Insert constraints

    @constraint(model, beta_abs .>= beta) #put the dot is like doing the loop over all j
    @constraint(model, beta_abs .>= -beta)
    
    
    #sum((y-X*beta).^2)) = sum((y[i] - sum(X[i,j]*beta[j]) for j=1:p)^2 for i=1:n)
    
    #Insert objective
    @objective(model, Min, sum((y-X*beta).^2) + lambda*sum(beta_abs))
    
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
    return (value.(beta))
    
end

lasso (generic function with 2 methods)

In [4]:
lasso_beta = lasso(X_train_solar,y_train_solar)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


26-element Vector{Float64}:
     3.2930604080946706
    -1.2645283093989246
     0.05795209313027566
     1.2743089330518962
   386.607099698167
   -11.05448733484336
   -11.950493209848675
     1.7392307161655942
     2.462835872195375
     0.05300911354446576
     3.6636030396839185
     0.32311800102602106
    -0.11588622639388668
     0.6170372687911923
   -15.018594243405959
    -2.3063739378878334
     1.2314555082385525
    -3.024275173659655
   -12.571435420678664
    64.37493246115822
     9.039641016224441e-8
   -44.002141802257334
 -1069.1089749681248
     4.271289781212573e-11
   -44.002141802257334
     9.039641016224441e-8

In [6]:
preds = X_valid_solar*lasso_beta;